# Group Surveys Feature Determination

## Preparation

### Import

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, QuantileTransformer
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 1000)

## Function Definition

### Min-Max Scaler

In [3]:
from sklearn.preprocessing import MinMaxScaler

def min_max_scaling_df(df):
    scaler = MinMaxScaler()
    scaled_df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
    return scaled_df


## Use of Function

In [4]:
in_file= r'/content/drive/MyDrive/Projects/tps/finals/data/4_individual_features.xlsx'
out_file= r'/content/drive/MyDrive/Projects/tps/finals/data/5_individual_features.xlsx'

In [5]:
df= pd.read_excel(in_file, index_col='Id')
df.head(12)

,group,indiv_spoken_time_ratio,average_turn_duration,avg_time_without_speaking,num_turns,num_turns_ratio,avg_turns_without_speaking,max_words_turn_ratio,text_joy,messages_sent,contribution_index,sentiment_avg,emotionality_avg,activity_entanglement,EMOTIONS_Happy,Groupflow_Beeflow,Groupflow_Leechflow,O,C,E,A,N,harm_care_score,fairness_reciprocity_score,in_group_loyality_score,authority_respect_score,purity_sanctity_score,theory,coeval,project
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,0.148336,7.850746,56.210746,67,0.331683,1.985075,0.078613,0.298809,34,-0.64,0.303561,0.215562,0.487649,0.407447,0.497477,0.156358,0.600000,0.716667,0.600000,0.633333,0.583333,27,23,19,18,20,3.44,0.90,8.0
2,1,0.746757,33.948718,21.791169,78,0.386139,1.589744,0.720231,0.292950,75,-0.34,0.250396,0.236507,0.398994,0.417419,0.300955,0.220056,0.533333,0.666667,0.700000,0.616667,0.633333,22,28,11,7,11,3.64,0.91,8.0
3,1,0.029611,8.076923,57.649231,13,0.064356,6.461538,0.080925,0.297727,38,-0.60,0.381860,0.254014,0.501416,0.410218,0.226931,0.238211,0.566667,0.683333,0.716667,0.533333,0.716667,23,28,9,6,7,3.98,0.90,8.0
4,1,0.045685,6.480000,159.421600,25,0.123762,6.600000,0.070520,0.300100,27,-0.70,0.250852,0.259828,0.493443,0.322770,0.256794,0.232465,0.566667,0.783333,0.733333,0.733333,0.633333,17,24,17,13,8,4.22,0.90,8.0
5,1,0.029611,5.526316,43.632632,19,0.094059,4.526316,0.049711,0.303593,22,-0.75,0.305600,0.244289,0.555323,0.472688,0.342162,0.120088,0.600000,0.666667,0.483333,0.583333,0.433333,12,15,11,19,4,3.74,0.91,8.0
6,2,0.337682,8.518987,19.887848,158,0.364055,1.740506,0.233503,0.297780,22,-0.36,0.404612,0.268756,0.520000,0.624326,0.332323,0.221886,0.683333,0.733333,0.883333,0.583333,0.416667,24,26,13,13,15,4.64,0.78,8.5
8,2,0.074762,11.461538,162.167200,26,0.059908,15.384615,0.169543,0.300151,8,-0.71,0.746794,0.579783,0.529475,0.685490,0.535406,0.037475,0.533333,0.683333,0.716667,0.500000,0.516667,28,20,21,23,24,3.61,0.78,8.5
9,2,0.174862,8.822785,47.935696,79,0.182028,4.455696,0.260914,0.295444,8,-0.71,0.428550,0.236425,0.518237,0.553516,0.065323,0.311680,0.616667,0.700000,0.716667,0.716667,0.550000,17,23,16,13,14,4.37,0.77,8.5
10,2,0.069242,6.272727,25.912727,44,0.101382,3.113636,0.079188,0.298211,24,-0.32,0.259263,0.253813,0.497027,0.602558,0.221334,0.277272,0.533333,0.700000,0.583333,0.550000,0.583333,27,26,18,18,18,5.33,0.90,8.5


In [6]:
def create_random_df(df):
  limit0= 1
  limit1= 9
  limit2= 17
  limit3= 26

  # Generate two random integers within the range 1-8
  range_1 = random.sample(range(limit0, limit1), 2)

  # Generate two random integers within the range 9-16
  range_2 = random.sample(range(limit1, limit2), 2)

  # Generate two random integers within the range 17-25
  range_3 = random.sample(range(limit2, limit3), 2)

  # Combine all six integers into a single list
  column_indexes = range_1 + range_2 + range_3

  new_df = df.iloc[:, column_indexes].copy()
  data = pd.concat([new_df, df['theory']], axis=1)
  data = min_max_scaling_df(data)
  selected_columns = df.columns[column_indexes].to_list()

  return data,selected_columns

In [7]:
#df= drop_bad_measured(df)


## Regression Models

In [8]:
def grid_search_cv(df):
  from sklearn.model_selection import GridSearchCV, cross_validate, train_test_split
  from sklearn.metrics import make_scorer, mean_squared_error, r2_score
  from sklearn.linear_model import LinearRegression
  from sklearn.tree import DecisionTreeRegressor
  from sklearn.ensemble import RandomForestRegressor

  # Split the data into training and test sets
  X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, :-1], df.iloc[:, -1], test_size=0.2, random_state=42)

  # Define the list of models to evaluate
  models = [
      ('Linear Regression', LinearRegression()),
      ('Decision Tree', DecisionTreeRegressor()),
      ('Random Forest', RandomForestRegressor())
  ]

  # Define the grid of hyperparameters for each model
  param_grid = {
      'Linear Regression': {},
      'Decision Tree': {'max_depth': [None, 5, 10]},
      'Random Forest': {'n_estimators': [100, 200, 300]}
  }

  # Define the scoring metrics for evaluation
  scoring = {
      'RMSE': make_scorer(mean_squared_error, squared=False),
      'R2': make_scorer(r2_score)
  }

  # Perform grid search and cross-validation for each model
  results = {}
  for name, model in models:
      grid_search = GridSearchCV(model, param_grid[name], scoring=scoring, refit='RMSE', cv=5)
      grid_search.fit(X_train, y_train)
      
      # Cross-validation scores for all parameter combinations
      cv_results = cross_validate(grid_search.best_estimator_, X_train, y_train, scoring=scoring, cv=5)
      
      results[name] = {
          'best_params': grid_search.best_params_,
          'best_estimator': grid_search.best_estimator_,
          'cv_RMSE': cv_results['test_RMSE'].mean(),
          'cv_R2': cv_results['test_R2'].mean()
      }

  # Print the results for all models and parameter combinations
  for name, result in results.items():
      print("Model:", name)
      print("Best Parameters:", result['best_params'])
      print("CV RMSE:", result['cv_RMSE'])
      print("CV R2:", result['cv_R2'])
      print()

  # Train the best model on the entire training dataset
  best_model = min(results, key=lambda x: results[x]['cv_RMSE'])
  final_model = results[best_model]['best_estimator']
  final_model.fit(X_train, y_train)

  # Evaluate the performance of the best model on the test set
  y_pred = final_model.predict(X_test)
  mse = mean_squared_error(y_test, y_pred)
  rmse = np.sqrt(mse)
  r2 = r2_score(y_test, y_pred)
  print("Best Model:", best_model)
  print("Test RMSE:", rmse)
  print("Test R2:", r2)


In [9]:
#df = df.drop(columns=['group','project', 'theory']) #'coeval', 'project','theory','group'

In [10]:
#grid_search_cv(df)

In [11]:
def try_regressions(df):

  # Step 2: Split the data into features (X) and the objective variable (y)
  X = df.iloc[:, :-1]  # All columns except the last one
  y = df.iloc[:, -1]   # Last column

  # Step 3: Split the data into training and testing sets
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

  # Step 4: Build and evaluate different OLS models
  models = []

  # Model 1: OLS using statsmodels
  model1 = sm.OLS(y_train, sm.add_constant(X_train))
  results1 = model1.fit()
  models.append(results1)

  # Model 2: OLS using scikit-learn (linear regression)
  from sklearn.linear_model import LinearRegression
  model2 = LinearRegression()
  model2.fit(X_train, y_train)
  models.append(model2)

  # Model 3: OLS using scikit-learn (Ridge regression)
  from sklearn.linear_model import Ridge
  model3 = Ridge(alpha=1.0)  # You can adjust the regularization parameter (alpha) as needed
  model3.fit(X_train, y_train)
  models.append(model3)

  # Step 5: Evaluate the models on the testing set
  for model in models:
      if isinstance(model, sm.regression.linear_model.RegressionResultsWrapper):
          X_test_with_constant = sm.add_constant(X_test)
          y_pred = model.predict(X_test_with_constant)
          r2 = model.rsquared
      else:
          y_pred = model.predict(X_test)
          r2 = model.score(X_test, y_test)

      # Calculate evaluation metrics (e.g., mean squared error)
      mse = np.mean((y_pred - y_test) ** 2)

      # Print the evaluation results
      print(model)
      print("Mean Squared Error:", mse)
      print("R-squared:", r2)
      print()



In [12]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge

def try_regressions(df):
    # Step 2: Split the data into features (X) and the objective variable (y)
    X = df.iloc[:, :-1]  # All columns except the last one
    y = df.iloc[:, -1]   # Last column

    # Step 3: Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Step 4: Build and evaluate different OLS models
    models = []

    # Model 1: OLS using scikit-learn (linear regression)
    model1 = LinearRegression()
    model1.fit(X_train, y_train)
    models.append(('Linear Regression', model1))

    # Model 2: OLS using scikit-learn (Ridge regression)
    model2 = Ridge(alpha=1.0)  # You can adjust the regularization parameter (alpha) as needed
    model2.fit(X_train, y_train)
    models.append(('Ridge Regression', model2))

    # Step 5: Evaluate the models using cross-validation
    for name, model in models:
        scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')
        mse = -scores.mean()
        r2 = model.score(X_test, y_test)

        # Print the evaluation results
        print(name)
        print("Mean Squared Error:", mse)
        print("R-squared:", r2)
        print()


In [13]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge

def try_regressions(df):
    # Step 2: Split the data into features (X) and the objective variable (y)
    X = df.iloc[:, :-1]  # All columns except the last one
    y = df.iloc[:, -1]   # Last column

    # Step 3: Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Step 4: Build and evaluate different OLS models
    models = []

    # Model 1: OLS using scikit-learn (linear regression)
    model1 = LinearRegression()
    model1.fit(X_train, y_train)
    models.append(('Linear Regression', model1))

    # Model 2: OLS using scikit-learn (Ridge regression)
    model2 = Ridge(alpha=1.0)  # You can adjust the regularization parameter (alpha) as needed
    model2.fit(X_train, y_train)
    models.append(('Ridge Regression', model2))

    # Step 5: Evaluate the models using cross-validation
    results = {}

    for name, model in models:
        scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')
        mse = -scores.mean()
        r2 = model.score(X_test, y_test)

        # Store the evaluation results in a dictionary
        results[name] = {'Mean Squared Error': mse, 'R-squared': r2}

    return results


## Use of Functions

In [14]:
import pandas as pd
import numpy as np

results = []
indexes = []

for x in range(0, 5000):
    df_test, index = create_random_df(df)
    results.append(try_regressions(df_test))
    indexes.append(index)

data = []

for i, result in enumerate(results):
    for model_name, metrics in result.items():
        r2 = metrics['R-squared']
        mse = metrics['Mean Squared Error']
        data.append([model_name, r2, mse, indexes[i]])

df_results = pd.DataFrame(data, columns=['Model', 'R-squared', 'Mean Squared Error', 'Index'])

df_results = df_results.sort_values(by='R-squared', ascending=False)

df_results.head(10)

,Model,R-squared,Mean Squared Error,Index
5596,Linear Regression,0.669857,0.059031,"[num_turns, indiv_spoken_time_ratio, activity_..."
4650,Linear Regression,0.635983,0.037089,"[num_turns, average_turn_duration, activity_en..."
772,Linear Regression,0.632904,0.051780,"[num_turns, text_joy, emotionality_avg, activi..."
424,Linear Regression,0.618783,0.055156,"[max_words_turn_ratio, num_turns, contribution..."
8198,Linear Regression,0.617097,0.106106,"[num_turns, num_turns_ratio, sentiment_avg, ac..."
6564,Linear Regression,0.611049,0.036578,"[text_joy, num_turns_ratio, messages_sent, con..."
6816,Linear Regression,0.603842,0.054127,"[avg_time_without_speaking, num_turns, activit..."
9536,Linear Regression,0.598687,0.030599,"[text_joy, indiv_spoken_time_ratio, contributi..."
8310,Linear Regression,0.593953,0.041740,"[num_turns, max_words_turn_ratio, EMOTIONS_Hap..."
5018,Linear Regression,0.587031,0.040421,"[num_turns, max_words_turn_ratio, activity_ent..."


In [15]:
df_results.head(5).to_excel(out_file, index=False)